In [1]:
import time
import math

import pandas as pd
import numpy as np

import torch as torch
import torch.utils.data as data
from torch.utils.data import Dataset

import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import itertools

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Описание выбранного подхода и краткий обзор других

Выбранный подход - Factorization Machines. Это модель машинного обучения, которая расширяет традиционную матричную факторизацию за счет изучения взаимодействий между различными значениями признаков модели. Преимуществом FM является то, что она решает проблему "холодного старта", можно делать прогнозы на основе метаданных пользователя (которые скорее всего есть в боевой системе), даже если это пользователь, которого система никогда раньше не видела. После обучения модели вектора могут уловить некую "семантику", как word2vec эмбеддинги, поэтому ожидается, что они могут показывать схожесть пользователей и фильмов.

Альтернативные подходы:
Методы матричной факторизации, такие как SVD, ALS, отпали, поскольку они требуют слишком много оперативной памяти, как для хранения таблицы со всеми данными, так и для вычисления матриц UV. Есть Incremental SVD, Incremental PCA, но в условии задачи также было **выделено** использование **нейросетевых подходов**.

Bert Embedding - мало данных и описательных характеристик фильмов и пользователей

# Обработка данных

In [3]:
# Загрузка полного датасета MovieLens
movies_df = pd.read_csv('/kaggle/input/grouplens-2018/ml-latest/movies.csv')
movies_df['movieId_index'] = movies_df['movieId'].astype('category').cat.codes

In [4]:
movies_df.head()

,movieId,title,genres,movieId_index
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,0
1,2,Jumanji (1995),Adventure|Children|Fantasy,1
2,3,Grumpier Old Men (1995),Comedy|Romance,2
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,3
4,5,Father of the Bride Part II (1995),Comedy,4


In [5]:
ratings = pd.read_csv('/kaggle/input/grouplens-2018/ml-latest/ratings.csv')
ratings = ratings.join(movies_df.set_index('movieId'),on='movieId')

In [6]:
ratings.head()

,userId,movieId,rating,timestamp,title,genres,movieId_index
0,1,307,3.5,1256677221,Three Colors: Blue (Trois couleurs: Bleu) (1993),Drama,304
1,1,481,3.5,1256677456,Kalifornia (1993),Drama|Thriller,477
2,1,1091,1.5,1256677471,Weekend at Bernie's (1989),Comedy,1069
3,1,1257,4.5,1256677460,Better Off Dead... (1985),Comedy|Romance,1229
4,1,1449,4.5,1256677264,Waiting for Guffman (1996),Comedy,1414


In [7]:
# Столбцы, которые будут использоваться как признаки
feature_columns = ['userId','movieId_index']

In [8]:
# Количество уникальных фильмов
n_movie_unique = len(ratings['movieId_index'].unique())
n_movie_unique

53889

In [9]:
# Количество уникальных пользователей
n_user_unique = len(ratings['userId'].unique())
n_user_unique

283228

In [11]:
# Получает "размер" идентификаторов
features_sizes = {
    'userId': n_user_unique,
    'movieId_index': n_movie_unique,
}

# Считает смещение идентификатора, 
# поскольку всё в одном пространстве.

# Следующий ид признака идет сразу после предыдущего
next_offset = 0
features_offsets={}
for k,v in features_sizes.items():
    features_offsets[k] = next_offset
    next_offset += v

In [12]:
# Смещает все индексы в датасете
for column in feature_columns:
    ratings[column] = ratings[column].apply(lambda c: c + features_offsets[column])   

In [13]:
ratings[[*feature_columns,'rating']].head()

,userId,movieId_index,rating
0,1,283532,3.5
1,1,283705,3.5
2,1,284297,1.5
3,1,284457,4.5
4,1,284642,4.5


In [14]:
data_x = torch.tensor(ratings[feature_columns].values)    # userId, movieId
data_y = torch.tensor(ratings['rating'].values).float()   # rating - таргет
dataset = data.TensorDataset(data_x, data_y)

In [15]:
bs = 1024                          # Размер батча
train_n = int(len(dataset)*0.9)    # Размер тренировочной выборки
valid_n = len(dataset) - train_n   # Размер валидационной выборки
splits = [train_n, valid_n]

trainset, devset = torch.utils.data.random_split(dataset, splits)
train_dataloader = data.DataLoader(trainset, batch_size=bs, shuffle=True)
dev_dataloader = data.DataLoader(devset, batch_size=bs, shuffle=True)

In [17]:
# Взята из fast.ai
# Эта функция инициализирует тензор x со значениями, 
# распределенными по нормальному закону с заданным средним значением и стандартным отклонением.
# Однако, она также обрезает значения, которые находятся за пределами двух стандартных отклонений от среднего, 
# чтобы избежать влияния выбросов на инициализацию.
def trunc_normal_(x, mean=0., std=1.):
    "Truncated normal initialization."
    return x.normal_().fmod_(2).mul_(std).add_(mean)

# Обучение модели

In [18]:
class FMModel(nn.Module):
    def __init__(self, n, k):
        super().__init__()

        self.w0 = nn.Parameter(torch.zeros(1))
        self.bias = nn.Embedding(n, 1)
        
        # При передаче индекса в Embedding слой, 
        # он использует этот индекс для извлечения соответствующего вектора из матрицы 
        # и возвращает его
        self.embeddings = nn.Embedding(n, k)

        with torch.no_grad(): trunc_normal_(self.embeddings.weight, std=0.01)
        with torch.no_grad(): trunc_normal_(self.bias.weight, std=0.01)

    def forward(self, X):
        emb = self.embeddings(X)
        pow_of_sum = emb.sum(dim=1).pow(2)
        sum_of_pow = emb.pow(2).sum(dim=1)
        pairwise = (pow_of_sum-sum_of_pow).sum(1) * 0.5
        bias = self.bias(X).squeeze().sum(1)
        return torch.sigmoid(self.w0 + bias + pairwise) * 5.5

In [19]:
# Обучает модель
def fit(iterator, model, optimizer, criterion):
    train_loss = 0
    model.train()
    for x,y in iterator:
        optimizer.zero_grad()
        y_hat = model(x.to(device))
        loss = criterion(y_hat, y.to(device))
        train_loss += loss.item()*x.shape[0]
        loss.backward()
        optimizer.step()
    return train_loss / len(iterator.dataset)

# Валидирует модель
def test(iterator, model, criterion):
    train_loss = 0
    model.eval()
    for x,y in iterator:                    
        with torch.no_grad():
            y_hat = model(x.to(device))
        loss = criterion(y_hat, y.to(device))
        train_loss += loss.item()*x.shape[0]
    return train_loss / len(iterator.dataset)

Выбор MSE в качестве лосс функции:

Сам по себе MSE измеряет близость между векторами, а также относительное расположение векторов в пространстве, что отлично подходит для задачи генерации эмбеддингов. 

In [20]:
# Проходит по эпохам обучения
def train_n_epochs(model, n, optimizer,scheduler):
    criterion = nn.MSELoss().to(device)
    for epoch in range(n):
        start_time = time.time()
        train_loss = fit(train_dataloader, model, optimizer, criterion)
        valid_loss = test(dev_dataloader, model, criterion)
        scheduler.step()
        secs = int(time.time() - start_time)
        print(f'epoch {epoch}. time: {secs}[s]')
        print(f'\ttrain rmse: {(math.sqrt(train_loss)):.4f}')
        print(f'\tvalidation rmse: {(math.sqrt(valid_loss)):.4f}')

Использование weight decay: В оптимизаторе Adam weight decay добавляется как L2-регуляризация к градиентам весов при обновлении параметров модели. Это позволяет уменьшить значения весов на каждой итерации обучения и снизить их влияние на функцию потерь. Мотивацией для выбора является неприлично огромный датасет.

Использование Scheduler для шагов оптимизатора: использование Scheduler для изменения learning rate в процессе обучения позволяет более точно настроить скорость обучения для каждой фазы обучения, что может улучшить качество модели за счет быстрого обучения в начале и более точного обновления весов в более поздних эпохах.


In [21]:
model = FMModel(data_x.max()+1, 120).to(device)
wd = 1e-5    # Weight decay для оптимизатора
lr = 0.001
epochs=10
optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=wd)
scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[7], gamma=0.1)
criterion = nn.MSELoss().to(device)
for epoch in range(epochs):
    start_time = time.time()
    train_loss = fit(train_dataloader, model, optimizer, criterion)
    valid_loss = test(dev_dataloader, model, criterion)
    scheduler.step()
    secs = int(time.time() - start_time)
    print(f'epoch {epoch}. time: {secs}[s]')
    print(f'\ttrain rmse: {(math.sqrt(train_loss)):.4f}')
    print(f'\tvalidation rmse: {(math.sqrt(valid_loss)):.4f}')

epoch 0. time: 589[s]
	train rmse: 0.8816
	validation rmse: 0.8576
epoch 1. time: 590[s]
	train rmse: 0.8513
	validation rmse: 0.8513
epoch 2. time: 590[s]
	train rmse: 0.8467
	validation rmse: 0.8485
epoch 3. time: 590[s]
	train rmse: 0.8444
	validation rmse: 0.8468
epoch 4. time: 590[s]
	train rmse: 0.8428
	validation rmse: 0.8455
epoch 5. time: 591[s]
	train rmse: 0.8416
	validation rmse: 0.8447
epoch 6. time: 589[s]
	train rmse: 0.8408
	validation rmse: 0.8442
epoch 7. time: 590[s]
	train rmse: 0.8301
	validation rmse: 0.8392
epoch 8. time: 591[s]
	train rmse: 0.8270
	validation rmse: 0.8371
epoch 9. time: 592[s]
	train rmse: 0.8249
	validation rmse: 0.8357


In [22]:
movies = ratings.drop_duplicates('movieId_index').copy()
movie_embeddings = model.embeddings(torch.tensor(movies['movieId_index'].values,device=device).long())
movies['embedding'] = movie_embeddings.tolist()
movie_biases = model.bias(torch.tensor(movies['movieId_index'].values,device=device).long())
movies['bias'] = movie_biases.cpu().detach().numpy()

In [23]:
movies[['title','movieId_index','embedding','bias']]

,title,movieId_index,embedding,bias
0,Three Colors: Blue (Trois couleurs: Bleu) (1993),283532,"[0.07495241612195969, 0.015841152518987656, -0...",4.716378e-01
1,Kalifornia (1993),283705,"[-0.04083696007728577, 0.006113792769610882, 0...",1.206774e-02
2,Weekend at Bernie's (1989),284297,"[-0.0346284843981266, -0.00011565708700800315,...",-3.358779e-01
3,Better Off Dead... (1985),284457,"[0.16088442504405975, -0.0072015016339719296, ...",3.801672e-01
4,Waiting for Guffman (1996),284642,"[0.1197023019194603, 0.003787460969761014, -0....",4.262060e-01
...,...,...,...,...
27680666,Stranglehold (1994),329727,"[0.00303276046179235, 0.0014628081116825342, 0...",-3.169615e-03
27708795,The Great Houdini (1976),326846,"[-3.918913324284713e-40, -5.727148862649457e-4...",7.084012e-40
27734907,Hotline (2014),309039,"[-7.177240539502064e-40, -3.3268507101228347e-...",6.393074e-40
27734957,Barnum! (1986),314890,"[2.3074761422497465e-40, 3.6391020469283337e-4...",6.807718e-40


In [24]:
movies[movies.movieId == 1]

,userId,movieId,rating,timestamp,title,genres,movieId_index,embedding,bias
42,4,1,4.0,1113765937,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,283228,"[-0.43275585770606995, 0.03316543251276016, -0...",0.393278


# Применение модели

### Поиск похожих фильмов

In [25]:
toy_story_index = torch.tensor(283228).to(device)
toy_story_embedding = model.embeddings(toy_story_index)
cosine_similarities = torch.tensor([F.cosine_similarity(toy_story_embedding,i,dim=0) for i in movie_embeddings])
movies.iloc[cosine_similarities.argsort(descending=True).detach().numpy()]['title'].values[:10]

array(['Toy Story (1995)', 'Toy Story 2 (1999)', 'Toy Story 3 (2010)',
       'Finding Nemo (2003)', 'Monsters, Inc. (2001)',
       "Bug's Life, A (1998)", 'Ratatouille (2007)',
       'Incredibles, The (2004)', 'Toy Story That Time Forgot (2014)',
       'Up (2009)'], dtype=object)

### Генерация рекомендаций пользователю

In [30]:
# Для генерации рекомендаций новому пользователю, 
# берем его фильмы и оценки и складываем в "мега"-эмбеддинг,
# а затем проводим поиск самых схожих среди всех фильмов.

rated_movies = [283697,   # In the Army Now (1994)
                283811,   # Terminator 2: Judgment Day (1991)
                283391]   # Die Hard: With a Vengeance (1995)

user_rate = 0.2 # 4/5 каждому фильму

mega_embedding = 0

for movie in rated_movies:
    mega_embedding += user_rate*(model.embeddings(torch.tensor(movie,device=device)))
    
rankings = movie_biases.squeeze()+(mega_embedding*movie_embeddings).sum(1)
[i for i in movies.iloc[rankings.argsort(descending=True).cpu()]['title'].values][:10]

['Terminator 2: Judgment Day (1991)',
 'Die Hard (1988)',
 'Jurassic Park (1993)',
 'Terminator, The (1984)',
 'Matrix, The (1999)',
 'True Lies (1994)',
 'Die Hard: With a Vengeance (1995)',
 'Independence Day (a.k.a. ID4) (1996)',
 'Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981)',
 'Back to the Future (1985)']

# Перспективы и что можно было сделать лучше

## Источники

1. Rendle S. Factorization machines //2010 IEEE International conference on data mining. – IEEE, 2010. – С. 995-1000. - а также github с имплементацией идей статьи: https://github.com/yonigottesman/recommendation_playground/blob/master/fm_movies.ipynb
2. https://blog.fastforwardlabs.com/2018/04/10/pytorch-for-recommenders-101.html - обзор и сравнение Matrix Factorization, Dense Feedforward NN, Sequence based RecSys
3. https://www.linkedin.com/pulse/how-implement-recommendation-system-deep-learning-pytorch-zaitsev - разбор на PyTorch реализации fast.ai RecSys
4. https://github.com/rposhala/Recommender-System-on-MovieLens-dataset#recommender-system-using-softmax-deep-neural-networks - Recommender System using Softmax Deep Neural Networks на TensorFlow